In [33]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

from t3_model import T3
from data_generator import DataGenerator

# Fill in the user parameters

In [17]:
# Data input file (must be CSV)
INPUT_DATA_FILENAME = "sample-data.csv"

# names of the input data columns as they appear in the CSV
FEATURE_COLS = ['PM1', 'PM2_5', 'PM10', 'Temperature', 'Humidity', 'RED', 'NOX'] 

# names of the target column[s] as they appear in the CSV
TARGET_COLS = ['OZONE']

# function to preprocess the data before it is converted into samples. 
#  Should take a Pandas DataFrame and return a new DataFrame
PREPROCESSOR = None

# function to normalize the dataset. Default is MinMaxScaler
SCALER = MinMaxScaler()

# number of rows per sample (the temporal window)
WINDOW = 100

# Periodicity, defined in seconds. Used to make sure there aren't any breaks in the sample 
PERIODICITY = 1 * 3600

# Whether to use sample renormalization (concatenate features renormalized for the sample window)
SAMPLE_RENORMALIZATION = True

# The renormalization kernel to use if SAMPLE_RENORMALIZATION is True
SAMPLE_RENORM_KERNEL = MinMaxScaler

# Training batch size
BATCH_SIZE = 32

# Length of the embedding vector
D_MODEL = 512

# Number of neurons in the feed-forward network
DFF = D_MODEL * 4

# Number of Multi-Attention Heads
NUM_HEADS = 1

# Width of the head
HEAD_SIZE = 128

# Number of Multi-Attention layers in series
NUM_LAYERS = 1

# Dropout rate
DROPOUT_RATE = 0.8

# Number of epochs to train over
NUM_EPOCHS = 100


# Create the Data Pool 

Your data should be stored in a `.csv` file with the following properties:

- The first row are headings (timestamp, pm2.5, o3)
- The first column are valid timestamps

In [29]:
# Read in the data to a Pandas DataFrame
data = pd.read_csv(INPUT_DATA_FILENAME, index_col=0, parse_dates=True)

# Preprocess the data if user supplied a function
if PREPROCESSOR:
    data = PREPROCESSOR(data)
    
# Normalize each data column to the appropriate range
data.loc[:, FEATURE_COLS + TARGET_COLS] = SCALER.fit_transform(data.loc[:, FEATURE_COLS + TARGET_COLS].values)
SCALER.feature_cols = FEATURE_COLS
SCALER.target_cols = TARGET_COLS

# Create the data pool
SAMPLES = []
expected_timespan = (WINDOW - 1) * PERIODICITY
num_skipped = 0
for i in range(len(data) - WINDOW):
    
    print(f'Parsed {i}/{len(data)} samples...', end='\r')

    x = data.iloc[i:i+WINDOW].copy()

    sample_timespan = int((x.iloc[-1].name - x.iloc[0].name).total_seconds())

    if sample_timespan != expected_timespan:
        num_skipped += 1
        continue
        
    if SAMPLE_RENORMALIZATION:
        x2 = x.loc[:, FEATURE_COLS].copy()
        x2.loc[:, :] = SAMPLE_RENORM_KERNEL().fit_transform(x2.values)
        x = x.join(x2, rsuffix='_r')

    SAMPLES.append(x)
    
print(f'Created {len(SAMPLES)} from {len(data)} rows of data. {num_skipped} samples were skipped due to noncontinuous data')    

# Create the train-test split
ind = np.arange(len(SAMPLES))
np.random.shuffle(ind)
split_point = int(len(ind) * 0.7)

print('Creating the training dataset...')
TRAIN_GEN = DataGenerator(
    [SAMPLES[i] for i in ind[:split_point]],
    batch_size=BATCH_SIZE,
    feature_cols=FEATURE_COLS,
    target_cols=TARGET_COLS
)
print('Creating the testing dataset...')
TEST_GEN = DataGenerator(
    [SAMPLES[i] for i in ind[split_point:]],
    batch_size=BATCH_SIZE,
    feature_cols=FEATURE_COLS,
    target_cols=TARGET_COLS
)

print(f'Number of training samples: {int(len(TRAIN_GEN) * 0.7)}')
print(f'Number of testing samples:  {int(len(TEST_GEN) * 0.3)}')

Created 9112 from 19328 rows of data. 10116 samples were skipped due to noncontinuous data
Creating the training dataset...
Creating the testing dataset...
Number of training samples: 199
Number of testing samples:  85


# Train the model

In [42]:
MODEL = T3(
    d_model=D_MODEL,
    dff=DFF,
    num_heads=NUM_HEADS,
    head_size=HEAD_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT_RATE,
    num_outputs=len(TARGET_COLS),
    final_activation='linear'
)

MODEL_DIR = 'model'

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

callbacks = []

checkpoints_cb = tf.keras.callbacks.ModelCheckpoint(
            filepath=f'{MODEL_DIR}/checkpoints',
            save_weights_only=False,
            monitor='val_loss',
            mode='auto',
            save_best_only=True,
            save_freq='epoch',
            verbose=1,
        )
callbacks.append(checkpoints_cb)

MODEL.compile(
        loss='mean_squared_error',
        optimizer='adam',
    )

MODEL.fit(
    x               = TRAIN_GEN,
    epochs          = NUM_EPOCHS,
    callbacks       = callbacks,
    validation_data = TEST_GEN,
    verbose         = 2,
    shuffle         = True
    )

InputEncodingLayer: (32, 100, 512)
Epoch 1/100
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)

Epoch 00001: val_loss improved from inf to 0.14967, saving model to model/checkpoints
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)


INFO:tensorflow:Assets written to: model/checkpoints/assets


INFO:tensorflow:Assets written to: model/checkpoints/assets


199/199 - 21s - loss: 7.5891 - val_loss: 0.1497 - 21s/epoch - 106ms/step
Epoch 2/100

Epoch 00002: val_loss improved from 0.14967 to 0.08025, saving model to model/checkpoints
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)


INFO:tensorflow:Assets written to: model/checkpoints/assets


...

INFO:tensorflow:Assets written to: model/checkpoints/assets


199/199 - 20s - loss: 0.0039 - val_loss: 0.0024 - 20s/epoch - 103ms/step
Epoch 97/100

Epoch 00097: val_loss did not improve from 0.00243
199/199 - 17s - loss: 0.0034 - val_loss: 0.0061 - 17s/epoch - 87ms/step
Epoch 98/100

Epoch 00098: val_loss improved from 0.00243 to 0.00235, saving model to model/checkpoints
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)
InputEncodingLayer: (None, 100, 512)


INFO:tensorflow:Assets written to: model/checkpoints/assets


INFO:tensorflow:Assets written to: model/checkpoints/assets


199/199 - 20s - loss: 0.0036 - val_loss: 0.0023 - 20s/epoch - 98ms/step
Epoch 99/100

Epoch 00099: val_loss did not improve from 0.00235
199/199 - 17s - loss: 0.0034 - val_loss: 0.0038 - 17s/epoch - 87ms/step
Epoch 100/100

Epoch 00100: val_loss did not improve from 0.00235
199/199 - 17s - loss: 0.0032 - val_loss: 0.0050 - 17s/epoch - 86ms/step
